# Dual Moving Average (Trend Following) — 2800.HK

- Idea: trend-following with fast/slow moving average crossover.
- Backtest: close-to-close; transaction cost charged on position changes (bps).

This is a classic trend baseline often used for index/ETF instruments.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fyp_trading.data import fetch_prices
from fyp_trading.backtest import equity_and_stats_from_positions
from fyp_trading.strategies import DualMaConfig, dual_ma_trend_signal

TICKER = "2800.HK"
PERIOD = "3y"
INTERVAL = "1d"
TRANSACTION_COST_BP = 2.0
ALLOW_SHORT = False

CFG = DualMaConfig(fast=10, slow=50, allow_short=ALLOW_SHORT)

df = fetch_prices(TICKER, PERIOD, INTERVAL)
close = df["Close"].copy()

pos = dual_ma_trend_signal(close, CFG)

# close-to-close daily return
next_ret = np.log(close.shift(-1) / close)
simple_ret = (np.exp(next_ret) - 1.0).dropna()

bt_df, stats = equity_and_stats_from_positions(
    simple_return=simple_ret,
    position=pos.loc[simple_ret.index],
    transaction_cost_bp=TRANSACTION_COST_BP,
)

print(pd.Series(stats))

plt.figure(figsize=(12,4))
plt.plot(bt_df["strategy_equity"], label="Dual-MA strategy")
plt.plot(bt_df["buyhold_equity"], label="Buy&Hold", linestyle="--")
plt.title(f"Dual MA Backtest | {TICKER} | fast={CFG.fast}, slow={CFG.slow}, cost={TRANSACTION_COST_BP:.0f}bps")
plt.legend()
plt.grid(alpha=0.3)
plt.show()
